In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import json
pd.set_option('display.max_columns', None)

In [2]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../Data-and-DBs/pokedex.db")

cursor = con.cursor()
cursor.execute('DROP table IF EXISTS logistic_regression_results')
cursor.execute('DROP table IF EXISTS logistic_regression_classifications')
cursor.execute('DROP table IF EXISTS logistic_regression_matrix')

train_df = pd.read_sql_query("SELECT * from gens_1_to_6", con)

# Verify that result of SQL query is stored in the dataframe
train_df.head()

,NUMBER,CODE,SERIAL,NAME,TYPE1,TYPE2,COLOR,ABILITY1,ABILITY2,ABILITY_HIDDEN,GENERATION,LEGENDARY_FLAG,HEIGHT,WEIGHT,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,TOTAL,CAPTURE_RATE
0,1,1,11,Bulbasaur,Grass,Poison,Green,Overgrow,None,Chrolophyll,1,0,0.7,6.9,45,49,49,65,65,45,318,45
1,2,1,21,Ivysaur,Grass,Poison,Green,Overgrow,None,Chrolophyll,1,0,1.0,13.0,60,62,63,80,80,60,405,45
2,3,1,31,Venusaur,Grass,Poison,Green,Overgrow,None,Chrolophyll,1,0,2.0,100.0,80,82,83,100,100,80,525,45
3,3,2,32,Mega Venusaur,Grass,Poison,Green,Thick Fat,None,None,1,0,2.4,155.5,80,100,123,122,120,80,625,45
4,4,1,41,Charmander,Fire,None,Red,Blaze,None,Solar Power,1,0,0.6,8.5,39,52,43,60,50,65,309,45


In [3]:
from sklearn.preprocessing import LabelEncoder
#encoding the training data
le = LabelEncoder()
encoded_training = train_df[train_df.columns[:]].apply(le.fit_transform)

In [4]:
encoded_training.head()

,NUMBER,CODE,SERIAL,NAME,TYPE1,TYPE2,COLOR,ABILITY1,ABILITY2,ABILITY_HIDDEN,GENERATION,LEGENDARY_FLAG,HEIGHT,WEIGHT,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,TOTAL,CAPTURE_RATE
0,0,0,0,67,9,14,3,82,53,11,0,0,6,49,17,24,23,41,37,24,57,5
1,1,0,1,294,9,14,3,82,53,11,0,0,9,88,31,37,37,55,50,38,94,5
2,2,0,2,720,9,14,3,82,53,11,0,0,19,318,51,57,55,71,66,58,164,5
3,2,1,3,435,9,14,3,138,53,70,0,0,23,358,51,73,86,86,77,58,187,5
4,3,0,4,86,6,12,7,9,53,107,0,0,5,63,11,27,18,36,22,43,51,5


In [5]:
# Assign X (data) and y (target) for train data
X = encoded_training.drop(["NUMBER", "CODE", "SERIAL", "NAME", "COLOR", "GENERATION", "HEIGHT", "WEIGHT", "LEGENDARY_FLAG"], axis=1)
y = encoded_training["LEGENDARY_FLAG"]
print(X.shape, y.shape)

(817, 13) (817,)


In [6]:
#call all entries from the gen7 and gen_8 table in the db
test_1_df = pd.read_sql('SELECT * FROM gen_7', con)
test_2_df = pd.read_sql('SELECT * FROM gen_8', con)

In [7]:
#encoding the test 1 & 2 data 
encoded_test_1 = test_1_df[test_1_df.columns[:]].apply(le.fit_transform)
encoded_test_2 = test_2_df[test_2_df.columns[:]].apply(le.fit_transform)

In [8]:
encoded_test_1.head()

,NUMBER,CODE,SERIAL,NAME,TYPE1,TYPE2,COLOR,ABILITY1,ABILITY2,ABILITY_HIDDEN,GENERATION,LEGENDARY_FLAG,HEIGHT,WEIGHT,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,TOTAL,CAPTURE_RATE
0,0,1,0,73,1,13,0,20,2,33,0,0,2,12,2,12,3,2,3,26,5,19
1,1,1,1,72,1,13,0,20,2,33,0,0,6,45,28,21,22,6,24,29,25,12
2,2,1,2,71,3,15,2,54,7,14,0,0,6,40,19,28,10,29,26,46,38,7
3,3,1,3,79,10,16,8,49,7,26,0,0,6,52,12,24,31,0,3,12,12,19
4,4,1,4,80,10,16,1,49,7,26,0,0,11,60,28,33,43,2,16,24,31,9


In [9]:
encoded_test_2.head()

,NUMBER,CODE,SERIAL,NAME,TYPE1,TYPE2,COLOR,ABILITY1,ABILITY2,ABILITY_HIDDEN,GENERATION,LEGENDARY_FLAG,HEIGHT,WEIGHT,HP,ATK,DEF,SP_ATK,SP_DEF,SPD,TOTAL,CAPTURE_RATE
0,0,2,0,58,16,10,2,35,26,43,0,0,3,20,9,16,9,7,4,10,11,15
1,1,1,1,71,14,10,8,44,15,0,0,0,7,40,9,23,9,20,17,36,25,12
2,2,1,2,73,14,3,8,44,15,0,0,0,16,70,17,29,15,26,23,42,42,4
3,3,1,3,89,14,10,5,16,14,30,0,0,11,46,26,16,13,7,4,1,17,12
4,4,1,4,87,13,13,5,40,14,30,0,0,15,66,29,29,20,33,20,6,40,6


In [10]:
#pull necessary info from the gen_7 and gen_8 testing dfs to create test data
X_test_1 = encoded_test_1.drop(["NUMBER", "CODE", "SERIAL", "NAME", "COLOR", "GENERATION", "HEIGHT", "WEIGHT", "LEGENDARY_FLAG"], axis=1)                     
y_test_1 = encoded_test_1['LEGENDARY_FLAG']

X_test_2 = encoded_test_2.drop(["NUMBER", "CODE", "SERIAL", "NAME", "COLOR", "GENERATION", "HEIGHT", "WEIGHT", "LEGENDARY_FLAG"], axis=1)                     
y_test_2 = encoded_test_2['LEGENDARY_FLAG']

#double-check shapes of X and y match
print(X_test_1.shape, y_test_1.shape)
print(X_test_2.shape, y_test_2.shape)

(118, 13) (118,)
(117, 13) (117,)


In [11]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [12]:
classifier.fit(X, y)

C:\Users\Kaylie\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
gen_7_score = classifier.score(X_test_1, y_test_1)
gen_8_score = classifier.score(X_test_2, y_test_2)

print(f"Training Data Score: {classifier.score(X, y)}")
print(f"Gen 7 Score: {gen_7_score}")
print(f"Gen 8 Score: {gen_8_score}")

Training Data Score: 0.9840881272949816
Gen 7 Score: 0.8220338983050848
Gen 8 Score: 0.9743589743589743


In [14]:
predictions_1 = classifier.predict(X_test_1)
predictions_2 = classifier.predict(X_test_2)
print(f"First 10 Predictions:   {predictions_1[:10]}")
print(f"First 10 Actual labels: {y_test_1[:10].tolist()}")
print(f"First 10 Predictions:   {predictions_2[:10]}")
print(f"First 10 Actual labels: {y_test_2[:10].tolist()}")

First 10 Predictions:   [0 0 0 0 0 0 0 0 0 0]
First 10 Actual labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
First 10 Predictions:   [0 0 0 0 0 0 0 0 1 1]
First 10 Actual labels: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [15]:
#calculate r2
from sklearn.metrics import r2_score

gen_7_r2 = r2_score(y_test_1, predictions_1)
gen_8_r2 = r2_score(y_test_2, predictions_2)

print('Gen 7 r2 ', gen_7_r2)
print('Gen 8 r2 ', gen_8_r2)

Gen 7 r2  0.08120133481646263
Gen 8 r2  0.8114930182599356


In [16]:
pd.DataFrame({"GEN 7 Prediction": predictions_1, "Actual": y_test_1}).reset_index(drop=True)

,GEN 7 Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
113,0,1
114,0,1
115,1,1
116,0,1


In [17]:
pd.DataFrame({"GEN 8 Prediction": predictions_2, "Actual": y_test_2}).reset_index(drop=True)

,GEN 8 Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
112,1,1
113,1,1
114,1,1
115,1,1


In [18]:
from sklearn.metrics import confusion_matrix

tn7, fp7, fn7, tp7 = confusion_matrix(y_test_1, predictions_1).ravel()
tn8, fp8, fn8, tp8 = confusion_matrix(y_test_2, predictions_2).ravel()

print(f'Gen 7 - tn: {tn7}, fp: {fp7}, fn: {fn7}, tp: {tp7}')
print(f'Gen 8 - tn: {tn8}, fp: {fp8}, fn: {fn8}, tp: {tp8}')

Gen 7 - tn: 87, fp: 0, fn: 21, tp: 10
Gen 8 - tn: 98, fp: 0, fn: 3, tp: 16


In [19]:
logistic_regression_matrix = pd.DataFrame({'Generation': ['Gen 7', 'Gen 8'],
                                           'TN': [tn7, tn8],
                                           'FP': [fp7, fp8], 
                                           'FN': [fn7, fn8], 
                                           'TP': [tp7, tp8]})
logistic_regression_matrix

,Generation,TN,FP,FN,TP
0,Gen 7,87,0,21,10
1,Gen 8,98,0,3,16


In [20]:
from sklearn.metrics import classification_report
Gen_7_class_report = classification_report(y_test_1, predictions_1, target_names=["Non-Legendary", "Legendary"], output_dict=True)
Gen_8_class_report = classification_report(y_test_2, predictions_2, target_names=["Non-Legendary", "Legendary"], output_dict=True)

In [21]:
print(f'Gen 7 Classification: {Gen_7_class_report}')
print(f'Gen 8 Classification: {Gen_8_class_report}')

Gen 7 Classification: {'Non-Legendary': {'precision': 0.8055555555555556, 'recall': 1.0, 'f1-score': 0.8923076923076924, 'support': 87}, 'Legendary': {'precision': 1.0, 'recall': 0.3225806451612903, 'f1-score': 0.4878048780487805, 'support': 31}, 'accuracy': 0.8220338983050848, 'macro avg': {'precision': 0.9027777777777778, 'recall': 0.6612903225806451, 'f1-score': 0.6900562851782364, 'support': 118}, 'weighted avg': {'precision': 0.856638418079096, 'recall': 0.8220338983050848, 'f1-score': 0.7860400038159443, 'support': 118}}
Gen 8 Classification: {'Non-Legendary': {'precision': 0.9702970297029703, 'recall': 1.0, 'f1-score': 0.9849246231155778, 'support': 98}, 'Legendary': {'precision': 1.0, 'recall': 0.8421052631578947, 'f1-score': 0.9142857142857143, 'support': 19}, 'accuracy': 0.9743589743589743, 'macro avg': {'precision': 0.9851485148514851, 'recall': 0.9210526315789473, 'f1-score': 0.949605168700646, 'support': 117}, 'weighted avg': {'precision': 0.9751205889819751, 'recall': 0.9

In [22]:
#convert results to dataframe
gen_7_classification_df = pd.DataFrame.from_dict(Gen_7_class_report)
gen_7_classification_df.rename({'precision': 'precision_gen_7', 
                                     'recall': 'recall_gen_7', 
                                     'f1-score': 'f1-score_gen_7', 
                                     'support': 'support_gen_7'}, inplace=True)
gen_7_classification_df.reset_index(inplace=True)
gen_7_classification_df = gen_7_classification_df.rename(columns = {'index':'attribute'})
gen_8_classification_df = pd.DataFrame.from_dict(Gen_8_class_report)
gen_8_classification_df.rename({'precision': 'precision_gen_8', 
                                     'recall': 'recall_gen_8', 
                                     'f1-score': 'f1-score_gen_8', 
                                     'support': 'support_gen_8'}, inplace=True)
gen_8_classification_df.reset_index(inplace=True)
gen_8_classification_df = gen_8_classification_df.rename(columns = {'index':'attribute'})
logistic_regression_classifications = gen_7_classification_df.append(gen_8_classification_df)
logistic_regression_classifications.rename(columns={'macro avg': 'macro_avg', 'weighted avg': 'weighted_avg'}, inplace=True)
logistic_regression_classifications = logistic_regression_classifications.round(3)
logistic_regression_classifications

,attribute,Non-Legendary,Legendary,accuracy,macro_avg,weighted_avg
0,precision_gen_7,0.806,1.000,0.822,0.903,0.857
1,recall_gen_7,1.000,0.323,0.822,0.661,0.822
2,f1-score_gen_7,0.892,0.488,0.822,0.690,0.786
3,support_gen_7,87.000,31.000,0.822,118.000,118.000
0,precision_gen_8,0.970,1.000,0.974,0.985,0.975
1,recall_gen_8,1.000,0.842,0.974,0.921,0.974
2,f1-score_gen_8,0.985,0.914,0.974,0.950,0.973
3,support_gen_8,98.000,19.000,0.974,117.000,117.000


In [23]:
logistic_regression_results = pd.DataFrame({'Generation': ['Gen 7', 'Gen 8'],
                           'Accuracy': [gen_7_score, gen_8_score],
                           'R2': [gen_7_r2, gen_8_r2]})
logistic_regression_results= logistic_regression_results.round(3)
logistic_regression_results

,Generation,Accuracy,R2
0,Gen 7,0.822,0.081
1,Gen 8,0.974,0.811


In [24]:
#convert results to sql table
logistic_regression_results.to_sql('logistic_regression_results', con, index=False)
logistic_regression_classifications.to_sql('logistic_regression_classifications', con, index=False)
logistic_regression_matrix.to_sql('logistic_regression_matrix', con, index=False)

In [25]:
con.close()